# Chapter 7: Direct Methods

In [1]:
import numpy as np
from copy import copy
import jax

## Algorithm 7.1

In [5]:
def basis(i,n):
    return [1.0 if k == i else 0.0 for k in range(n)]

## Algorithm 7.2

In [2]:
def cyclic_coordinate_descent(f, x, epsilon):
    delta, n = np.infty, x.shape[0]
    while np.abs(delta) > epsilon:
        x_prime = copy(x)
        for i in range(n):
            d = basis(i,n)
            x = line_search(f,x,d)
        delta = np.linalg.norm(x-x_prime)
    
    return x

### Example

In [25]:
def line_search(f, x, d):
    '''
    Gradient descent with armijo condition
    '''
    alpha = 1.0
    tau = 0.7
    c = 0.1
    gradient = jax.grad(f)

    while f(x  + alpha*np.array(d)) > f(x) + c*alpha*np.matmul( np.array(gradient(x)).reshape(-1,1).T , np.array(d).reshape(-1,1) )[0,0]:
        alpha = tau * alpha
    
    return x  + alpha*np.array(d) 

func = lambda x: (1-x[0])**2 + 5*(4*x[1] - x[0]**2)**2

x = np.array([1.0, 3.0])
epsilon = 0.01
x_sol = cyclic_coordinate_descent(func, x, epsilon)
print(x_sol)

[3.96834752 3.9401    ]


## Algorithm 7.3

In [27]:
def cyclic_coordinate_descent_with_acceleration_step(f, x, epsilon):
    delta, n = np.infty, x.shape[0]
    while np.abs(delta) > epsilon:
        x_prime = copy(x)
        for i in range(n):
            d = basis(i,n)
            x = line_search(f,x,d)
        x = line_search(f, x, x-x_prime)
        delta = np.linalg.norm(x-x_prime)
    
    return x

## Algorithm 7.4

In [32]:
def powell(f, x, epsilon):
    n = x.shape[0]
    U = [basis(i,n) for i in range(n)]
    delta = np.infty
    while delta > epsilon:
        x_prime = copy(x)
        for i in range(n):
            d = U[i]
            x_prime = line_search(f, x_prime, d)
        
        for i in range(n-1):
            U[i] = U[i+1]
        
        U[n-1] = d = x_prime - x
        x_prime = line_search(f, x ,d)
        delta = np.linalg.norm(x-x_prime)
        x = x_prime 
    
    return x